In [2]:
# This function generates robots as lists of their properties.
# Each robot has a unique name and 10 integers as their properties.
# @param n: number of robots
# @return: list with robots with name and properties

In [3]:
import random
def robot_generator(n):
    a = []
    for i in range(n):
        robot_gen = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        sum = 0
        max = 10
        pos = random.sample(range(0, 10), 10)
        for j in range(10):
            if j == 9:
                robot_gen[pos[j]] = max
                break
            randVar = random.randint(0,max)
            robot_gen[pos[j]] = randVar
            sum = sum + randVar
            max = max - randVar
        a.append(robot_gen)
    
    # unique naming
    for num in range(n):
        a[num] = ['r' + str(num), a[num]]
    
    return a

In [4]:
# This function teams up robots (generated with function robot_generator(n)).
# Every containes 10 robots.
# @param r: the robots which are generated in beforehand
# @param n: number of robots
# @return: a list with teams of 10 robots

In [5]:
def team_generator(r, n):
    teams = []
    
    # create unique sequence for picking robots
    r_id = random.sample(range(0,n), n)
    
    # pick 10 robots for each team
    idx = 0
    for i in range(int(n/10)):
        teams.append([r[r_id[idx+0]],r[r_id[idx+1]],r[r_id[idx+2]],r[r_id[idx+3]],r[r_id[idx+4]],
                      r[r_id[idx+5]],r[r_id[idx+6]],r[r_id[idx+7]],r[r_id[idx+8]],r[r_id[idx+9]]])
        idx = idx+10
    return teams

In [6]:
# This function creates different solutions as the population of the evolutionary algorithm.
# Solutions in this case are different partitions of the robot teams which are generated with
# the function team_generator(n)).
# @param r: the robots which are generated in beforehand
# @param n: number of robots
# @param m: number of solutions/partitions (also the size of the population)

In [7]:
def initial_population(r, n, m):  
    partitions = []
    for i in range(m):
        partitions.append(team_generator(r,n))
    return partitions

In [8]:
# This function calculates the fitness of each team in one solution.
# The fitness of each team is defined by the sum of the effective properties of its robots.
# The i-th effective property is defined as the highest property of all i-th robot properties.
# @param s: one solution (also partition)
# @return: list of fitness values for each team of the solution

In [9]:
def calculateTeamsFitness(s):
    fitness_teams = []
    # for each team
    for i in range(len(s)):
        sum = 0
        # for each robot
        for j in range(len(s[i])):
            temp = 0
            # for each robot property
            for k in range(len(s[i][j][1])):
                temp1 = s[i][k][1][j]
                if temp1>temp:
                    temp = temp1
            sum = sum + temp
        fitness_teams.append(sum)
    return fitness_teams

In [10]:
# This function calculates the fitness of each solution out of its fitness values for each team.
# It is defined as the average fitness value of its teams.
# @param partitions: list of all partitions (also population)
# @return: list with finess values of each partition/solution of the population

In [11]:
def calculateFitness(partitions):
    partitionFitnesses = []

    # for every solution/partition
    for p in range(len(partitions)):
        # init fitness list of teams
        teamsFitnesses = calculateTeamsFitness(partitions[p])
        
        # calculate the average
        tempPartitionSum = 0
        for i in range(len(partitions[p])):
            tempPartitionSum += teamsFitnesses[i]
        partitionFitness = tempPartitionSum/len(partitions[p])
        
        # add the average to the list
        partitionFitnesses.append(partitionFitness)
    
    return partitionFitnesses

In [12]:
# This function selects solutions with the roulette wheel selection method.
# According to the individual i's relative fitness it can be selected with the
# probability of pi = fi/sum of fj.
# @param parents: the solutions from which to pick individuals
# @param fitnessValues: list of fitness of each solution
# @param numberOfSelectedParents: the number of individuals to select
# @return: list of selected individuals

In [13]:
import random
def rouletteWheelSelection(parents, fitnessValues, numberOfSelectedParents):
    
    # sum of all fitness values
    fitnessSum = 0
    for fitness in range(len(fitnessValues)):
        fitnessSum += fitness
     
    # relative fitness of each individual resembles part of the roulette wheel
    relativeFitnesses = []#fitnessValues
    endOfLastRouletteWheelPart = 0
    for f in range(len(fitnessValues)):
        # covering the roulette wheel with probability intervals
        relativeFitnesses.append(endOfLastRouletteWheelPart + fitnessValues[f]/fitnessSum)
        endOfLastRouletteWheelPart += relativeFitnesses[f]

    # 'spinning' the roulette as often as we want to select a parent (numberOfSelectedParents)
    selectedParents = []
    for i in range(numberOfSelectedParents):

        # generate random number between highest relative fitness and 0 (point where 'ball' stops)
        randomNum = random.uniform(0, relativeFitnesses[len(parents)-1])
        a = []
        # check to which probability interval and parent it corresponds
        for p in range(len(parents)):
            if(randomNum <= relativeFitnesses[p]):
                # if already chosen
                #if(parents[p] in selectedParents):
                    # chose next parent not already in the list
                #    nextFound = False
                #    for nextId in range(len(parents)-p):
                #        nextPar = nextId + p
                #        if parents[nextPar] not in selectedParents:
                #            nextFound = True
                #            selectedParents.append(parents[nextPar])
                #            break
                #    if(nextFound):
                #        break
                #    else:
                        # search for previous parent not already in the list
                #        for prevId in range(p):
                #            descId = p-prevId
                #            if parents[descId] not in selectedParents:
                #                selectedParents.append(parents[descId])
                #                break
                #        break
                #else:
                    # if not already chosen chose this parent
                #    selectedParents.append(parents[p])
                #    break
                selectedParents.append(parents[p])
                break
    
    #print('len selected: ' + str(len(selectedParents)))
    # just to be sure
    #if(len(selectedParents) < numberOfSelectedParents):
    #    missing = numberOfSelectedParents-len(selectedParents)
    #    for i in range(missing):
    #        for par in parents:
    #            if(par not in selectedParents):
    #                selectedParents.append(par)
    #                break            
    #print('len selected: ' + str(len(selectedParents)))
    
    return selectedParents;

In [14]:
# This function recombines solutions (of the rouletteWheelSelection) with the one-point crossover method.
# It is swapping half of the teams of one solution with the other half of one another solution.
# The size of the input is equal to the size of the output.
# It also has to be an even number. Otherwise the crossover will not work.
# @param matchSet: the parents selected with the EA's selection method
# @return: list of recombined children

In [15]:
def crossover(matchSet):
    crossedPartitions = []
    
    # for half the length of the matchset
    matchSethalf = int(len(matchSet)/2)
    
    for j in range(matchSethalf):
        tmpPartition1Firsthalf = []
        tmpPartition2Firsthalf = []
        
        partitionHalf = int(len(matchSet[0])/2)
        
        # cross ith partition with the i+halfOfMatchset-th partition
        for i in range(partitionHalf):
            # store first half of one partition temporarily
            tmpPartition1Firsthalf.append(matchSet[j][i])
            tmpPartition2Firsthalf.append(matchSet[j+matchSethalf][i])

        for i in range(partitionHalf):
            # fill second half with remaining teams
            i = i + partitionHalf
            tmpPartition2Firsthalf.append(matchSet[j][i])
            tmpPartition1Firsthalf.append(matchSet[j+matchSethalf][i])

        crossedPartitions.append(tmpPartition1Firsthalf)
        crossedPartitions.append(tmpPartition2Firsthalf)
    
    return crossedPartitions

In [16]:
# This function repairs infeasible solutions that can rise from a crossover.
# Solutions of crossover are infeasible, if the solution contains duplicate robots
# in at least one partition/solution.
# @param solutions: the list children from the crossover
# @param parents: the list of the input parents for the crossover (individuals are all feasible)
# @return: the list of repaired children

In [17]:
def repair(solutions, parents):
    for child in solutions:
        a = []
        var = 0
        # for each team i
        for i in child:
            # for each robot in team i
            for j in i:
                # fill a with the robot, if not already contained
                if j not in a:
                    a.append(j)
                # if already in a, then find another robot from the matchSet
                else:
                    # for each team in one solution
                    for l in parents[0]:
                        # for each robot in that team
                        for m in l:
                            # if this robot is not already contained, add to a
                            if m not in a:
                                var = 1
                                a.append(m)
                                break
                        # break from loop/finding replacement for duplicate if robot is found
                        if var == 1:
                            var = 0
                            break 

        # update crossover solutions with repaired list a
        k = 0
        temp = []
        # for each team of the current partition
        for i in range(len(child)):
            # for each robot
            for j in range(len(child[0])):
                # add replacements to a temporary list
                temp.append(a[k])
                k = k + 1
            # partition i is updated with this list
            child[i] = temp
            temp = []
            
    return solutions

In [18]:
# This function mutates one solution.
# Mutation is defined by swapping 2 random robots in 2 random teams of one solution.
# @param partition: the solution/partition in which the mutation should take place
# @return: the mutated solution

In [19]:
import random
def mutation(partition):
    
    # select 2 random teams
    randTeam1 = random.randint(0,(len(partition)-1))
    randTeam2Found = False
    while(not randTeam2Found):
        randTeam2 = random.randint(0,(len(partition)-1))
        if(randTeam2 != randTeam1):
            randTeam2Found = True
    
    # generate 2 random positions in the teams
    randVar1 = random.randint(0,len(partition[0])-1)
    randVar2 = random.randint(0,len(partition[0])-1)
    
    # temporarily storing robots for swapping between teams
    temp1 = partition[randTeam1][randVar1]
    temp2 = partition[randTeam2][randVar2]
    
    # placing them in respected positions
    partition[randTeam1][randVar1] = temp2
    partition[randTeam2][randVar2] = temp1
    
    return partition

In [20]:
# This function selects the best individuals for the reproduction based on the (μ+λ)-scheme.
# In the μ+λ reproduction scheme the best μ ones from μ parents and λ children are selected.
# For finding the best ones, their fitness values (calculated via calculateFitness-function)
# are considered.
# @param parents: the parent solutions
# @param children: the crossed, repaired and mutated parents
# @return: the list of the best μ ones out of these two

In [21]:
def reproductionScheme(parents, children):
    selectedParFs = calculateFitness(parents)
    childrensFs = []
    for i in range(len(calculateFitness(children))):
        childrensFs.append(calculateFitness(children)[i])
    
    #print('parents fitness')
    #print(selectedParFs)
    #print('par len: ' + str(len(selectedParFs)))
    #print('childrens fitness')
    #print(childrensFs)
    #print('childs len: ' + str(len(childrensFs)))
    
    joined = []
    for i in range(len(parents)):
        joined.append([selectedParFs[i], parents[i]])
        joined.append([childrensFs[i], children[i]])
    
    descSortedJoined = sorted(joined, key=lambda x:float(x[0]), reverse=True)
    
    best = []
    for i in range(len(parents)):
        best.append(descSortedJoined[i][1])
    bestFs = calculateFitness(best)
    
    #print('best fitness')
    #print(bestFs)
    #print('best len: ' + str(len(best)))
    
    return best

In [22]:
# Test (reproductionScheme)
r1 = ['r1', [1,1]]
r2 = ['r2', [2,2]]
r3 = ['r3', [3,3]]
r4 = ['r4', [4,4]]
p  = [[r1, r1],[r1, r1]]
p2 = [[r3, r3],[r3, r3]]
c  = [[r3, r3],[r3, r3]]
c2 = [[r2, r4],[r2, r4]]
ps = [p, p2]
cs = [c, c2]
#reproductionScheme(ps, cs)

In [23]:
# This function averages the fitness values of one iteration of the EA.
# The fitness values are the values of the recombined and mutated parents plus the rest of the initial
# population for the current iteration.
# @param iterationFs: the list of fitness values of each solution in the population
# @return: the average of the values

In [24]:
def avgFitness(iterationFs):
    fitnessSum = 0
    for fitness in iterationFs:
        fitnessSum += fitness
        
    fitnessAvg = fitnessSum/len(iterationFs)
    
    return fitnessAvg

In [25]:
# This function reads .csv files of the given instances and converts them into the
# robot data structure where every robot has a name and its properties.
# @param filename: the name of the file as string
# @return: the list of robots conform with our EA

In [26]:
import pandas as pd
import csv
def convertInstancesToRobots(filename):
    rs = []
    rId = 0
    with open(filename) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=' ')
        for row in readCSV:
            prop = []
            for i in range(len(row)-1):
                prop.append(int(row[i]))

            rs.append(['r'+str(rId) , prop])
            rId = rId+1       
    return rs

In [27]:
#Pseudo Code: Evolutionary algorithm
# begin
#    t <- 0
#    1) initialize population
#    repeat
#       2) selection according to 3) fitness
#       4) crossover
#       5) mutation
#       6) update population
#       7) calculate average sum of fitness
#       t <- t+1
#    until 8) termination condition
# end

In [28]:
# This is the main function of the evolutionary algorithm (EA) (Pseudo Code is given above).
# @param r: robots generated in beforehand
# @param n: number of robots
# @param m: number of solutions (in this case partitions)
# @param numToSelect: number of parents to be selected for recombination/mutation
# @param numItr: number of iterations for the whole algorithm
# @output: logfile with parameters, results of every iteration and overall result

In [29]:
import csv
def main(r, n, m, numToSelect, numItr, logfileName):    
    #logFile = open(logfileName + '.txt', 'w')
    csvFile = open(logfileName + '.csv', 'w', newline='')
    csvWriter = csv.writer(csvFile)
    
    ## 1) initialize population ##
    population = initial_population(r, n, m)
    populationFs = calculateFitness(population)
    currBestAvgF = 0
    currBestS = []
    
    ## store values ##
    #logFile.write('Number of robots: ' + str(n) +' \n')
    #logFile.write('Robots: ' + str(r) +' \n')
    #logFile.write('Size of population: ' + str(m) +' \n')
    #logFile.write('Population: ' + str(population) +' \n')
    #logFile.write('Number of parents to recombine/mutate: ' + str(numToSelect) +' \n')
    #logFile.write('Number of iterations: ' + str(numItr) +' \n')
    
    csvWriter.writerow(['Number of robots', 'Populationsize', 'Number of parents', 'Number of iterations'])
    csvWriter.writerow([n, m, numToSelect, numItr])
    #csvWriter.writerow(r)
    zr = zip(r)
    for row in zr:
        csvWriter.writerow(row)
    #csvWriter.writerow(population)
    
    header = ['Iteration', 'Average iteration fitness']
    for i in range(m):
        header.append('Solution ' + str(i) + ' fitness')
    csvWriter.writerow(header)
    
    ## EA loop ##
    # 8) termination condition
    for iteration in range(numItr):
        #logFile.write('Initial fitness values in iteration ' + str(iteration) + ' :'+ str(populationFs) +' \n')
        
        # 2) selection according to 3) fitness
        selectedPar = rouletteWheelSelection(population, populationFs, numToSelect)
        
        # 4) crossover
        crossedPar = crossover(selectedPar)
        repairedPar = repair(crossedPar, selectedPar)
        
        # 5) mutation
        mutatedPar = []
        for i in range(len(repairedPar)):
            mutatedPar.append(mutation(repairedPar[i]))
        
        # 6) update population
        bestOffspring = reproductionScheme(selectedPar, mutatedPar)
        bestOffspringFs = calculateFitness(bestOffspring)
        
        idxList = []
        for i in selectedPar:
            idxList.append(population.index(i))
        for i in range(numToSelect):
            del population[idxList[i]]
            del populationFs[idxList[i]]
            population.append(bestOffspring[i])
            populationFs.append(bestOffspringFs[i])
        
        # 7) calculate average sum of fitness
        itrAvgF = avgFitness(populationFs)
        #logFile.write('Average fitness of iteration ' + str(iteration) + ' :' + str(itrAvgF) +' \n')
        
        # determine best overall solution
        for i in range(len(populationFs)):
            if(populationFs[i] > currBestAvgF):
                currBestAvgF = populationFs[i]
                currBestS = population[i]
        
        # store values in .csv
        itInformation = []
        itInformation.append(iteration)
        itInformation.append(float(itrAvgF))
        for i in populationFs:
            itInformation.append(i)
        csvWriter.writerow(itInformation)
      
    
    csvWriter.writerow(['Best solution average', 'Best solution', 'Best solution fitness values'])
    bestTFs = calculateTeamsFitness(currBestS)
    csvWriter.writerow([currBestAvgF, currBestS, bestTFs])
    #print("Best partition was: ")
    #print(currBestS)
    print("Best partition average fitness/effectiveness : ")
    print(currBestAvgF)
    
    #logFile.close()
    csvFile.close()
    
    return ''

In [30]:
# Output of the evolutionary algorithm:
# main(r, n, m, numToSelect, numItr, logfileName)

In [31]:
#robots = robot_generator(100)
#main(robots, 100, 8, 4, 50, 'EA_logFile.txt')

In [32]:
# This function executes the algorithm with different settings for the parameter that defines
# how many parents are selected for recombination/mutation.
# @param instanceName: the filename of the file containing an instance as string
# @param m: the number of solutions in the population (also population size)
# @param numItr: the number of iterations of the EA
# @return: an empty string for the results are saved in different .csv files

In [33]:
def executeEAforInstance(instanceName, m, numItr):
    r = convertInstancesToRobots(instanceName + '.csv')
    main(r, len(r), m,          m, numItr, 'EA_logFile_' + instanceName +'_100percent')# 100%
    main(r, len(r), m, int(m*0.9), numItr, 'EA_logFile_' + instanceName +'_90percent') #  90%
    main(r, len(r), m, int(m*0.8), numItr, 'EA_logFile_' + instanceName +'_80percent') #  80%
    main(r, len(r), m, int(m*0.5), numItr, 'EA_logFile_' + instanceName +'_50percent') #  50%
    main(r, len(r), m, int(m*0.3), numItr, 'EA_logFile_' + instanceName +'_30percent') #  30%
    return ''

In [34]:
executeEAforInstance('instance1', 20, 100)
executeEAforInstance('instance2', 20, 100)
executeEAforInstance('instance3', 20, 100)

Best partition average fitness/effectiveness : 
61.2
Best partition average fitness/effectiveness : 
60.0
Best partition average fitness/effectiveness : 
61.6
Best partition average fitness/effectiveness : 
61.6
Best partition average fitness/effectiveness : 
61.9
Best partition average fitness/effectiveness : 
72.0
Best partition average fitness/effectiveness : 
76.0
Best partition average fitness/effectiveness : 
74.0
Best partition average fitness/effectiveness : 
75.0
Best partition average fitness/effectiveness : 
76.0
Best partition average fitness/effectiveness : 
27.1
Best partition average fitness/effectiveness : 
27.0
Best partition average fitness/effectiveness : 
27.0
Best partition average fitness/effectiveness : 
26.8
Best partition average fitness/effectiveness : 
27.4


''

In [35]:
# This function plots the data of the executeEAforInstance-function as line charts.
# @param instanceName: the filename of the file containing an instance as string
# return: the graph/plot

In [36]:
#!pip install plotly
import csv

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout
init_notebook_mode(connected=True)

def plotInstance(instanceName):

    # init reader
    file = open('EA_logFile_' + instanceName +'_100percent.csv')
    fileReader = csv.reader(file)
    data = list(fileReader)
    file90 = open('EA_logFile_' + instanceName + '_90percent.csv')
    fileReader90 = csv.reader(file90)
    data90 = list(fileReader90)
    file80 = open('EA_logFile_' + instanceName + '_80percent.csv')
    fileReader80 = csv.reader(file80)
    data80 = list(fileReader80)
    file50 = open('EA_logFile_' + instanceName + '_50percent.csv')
    fileReader50 = csv.reader(file50)
    data50 = list(fileReader50)
    file30 = open('EA_logFile_' + instanceName + '_30percent.csv')
    fileReader30 = csv.reader(file30)
    data30 = list(fileReader30)

    # read data
    iteration = []
    fitness = []
    for i in range(int(data[1][3])):
        iteration.append(int(data[103+i][0]))
        fitness.append(float(data[103+i][1]))
    iteration90 = []
    fitness90 = []
    for i in range(int(data90[1][3])):
        iteration90.append(int(data90[103+i][0]))
        fitness90.append(float(data90[103+i][1]))
    iteration80 = []
    fitness80 = []
    for i in range(int(data80[1][3])):
        iteration80.append(int(data80[103+i][0]))
        fitness80.append(float(data80[103+i][1]))
    iteration50 = []
    fitness50 = []
    for i in range(int(data50[1][3])):
        iteration50.append(int(data50[103+i][0]))
        fitness50.append(float(data50[103+i][1]))
    iteration30 = []
    fitness30 = []
    for i in range(int(data30[1][3])):
        iteration30.append(int(data30[103+i][0]))
        fitness30.append(float(data30[103+i][1]))

    # Create and style traces
    trace0 = Scatter(
        x = iteration,
        y = fitness,
        name = '100% selected',
        line = dict(
            color = ('rgb(205, 12, 24)'),
            width = 2))
    trace1 = Scatter(
        x = iteration90,
        y = fitness90,
        name = '90% selected',
        line = dict(
            color = ('rgb(255, 144, 0)'),
            width = 2))
    trace2 = Scatter(
        x = iteration80,
        y = fitness80,
        name = '80% selected',
        line = dict(
            color = ('rgb(22, 96, 167)'),
            width = 2))
    trace3 = Scatter(
        x = iteration50,
        y = fitness50,
        name = '50% selected',
        line = dict(
            color = ('rgb(69, 144, 91)'),
            width = 2))
    trace4 = Scatter(
        x = iteration30,
        y = fitness30,
        name = '30% selected',
        line = dict(
            color = ('rgb(133, 56, 150)'),
            width = 2))

    # Edit the layout
    layout = dict(title = 'Convergence '+instanceName,
                  xaxis = dict(title = 'Iterations'),
                  yaxis = dict(title = 'Fitness'),
                  )

    fig = dict(data=[trace0, trace1, trace2, trace3, trace4], layout=layout)
    
    return iplot(fig, filename='ConvergencePlot')

In [37]:
plotInstance('instance1')

In [38]:
plotInstance('instance2')

In [39]:
plotInstance('instance3')